type- to jest dobre do przewidywania

In [1]:
import tensorflow as tf
import requests
import zipfile
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import seaborn as sb
# Note: this notebook requires torch >= 1.10.0
import keras
#device = "cuda" if torch.cuda.is_available() else "cpu"
#device

###Creating train and test dataset


In [2]:
def loadFromFile():
  datasetFolder = ('/content/drive/MyDrive/MLB Data')
  atBat = pd.read_csv(datasetFolder + '/atbats.csv')
  atBat['split'] = np.random.randn(atBat.shape[0], 1)
  players = pd.read_csv(datasetFolder + '/player_names.csv')
  pitches = pd.read_csv(datasetFolder + '/pitches.csv')
  pitches['split'] = np.random.randn(pitches.shape[0], 1)

  pitchAndAtBat = pd.merge(pitches, atBat, on='ab_id', how= 'left')

  allStatsPitchers= pd.merge(pitchAndAtBat, players, left_on='pitcher_id', right_on='id', how= 'left')
  pitches.columns
  allStatsPitchers.rename(columns = {'first_name':'pitch_first_name', 'last_name':'pitch_last_name'}, inplace = True)
  allStatsPitchers.drop(columns=['id'])

  allStats= pd.merge(allStatsPitchers, players, left_on='batter_id', right_on='id', how= 'left')
  allStats.rename(columns = {'first_name':'bat_first_name', 'last_name':'bat_last_name'}, inplace = True)
#allStats.drop(columns=['id'])
#allStats.dtypes


  allStats.columns
  allStats=allStats.dropna().reset_index(drop=True)
  return allStats


In [3]:
allStats = loadFromFile()

allStats.drop(['px','pz','bat_first_name','bat_last_name','ab_id','type_confidence','sz_top','sz_bot', 'nasty'], axis=1, inplace=True)
allStats.drop(['pitch_first_name','pitch_last_name','g_id','event_num','o'], axis=1, inplace=True)
allStats.drop(["batter_id", 'pitcher_id', 'break_angle',	'break_length'], axis=1, inplace=True)
allStats.drop(["split_x", 'split_y'], axis=1, inplace=True)
allStats.drop(['id_x', 'id_y','zone'], axis=1, inplace=True)

allStats.columns


Index(['start_speed', 'end_speed', 'spin_rate', 'spin_dir', 'break_y', 'ax',
       'ay', 'az', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0', 'z0', 'pfx_x',
       'pfx_z', 'code', 'type', 'pitch_type', 'b_score', 'b_count', 's_count',
       'outs', 'pitch_num', 'on_1b', 'on_2b', 'on_3b', 'event', 'inning',
       'p_score', 'p_throws', 'stand', 'top'],
      dtype='object')

In [4]:
allStats.dtypes
allStats['code']  = allStats['code'].astype(str)
allStats['stand'] = allStats['stand'].astype(str)
allStats['top'] = allStats['top'].astype(str)
allStats['type'] = allStats['type'].astype(str)
allStats['pitch_type']  = allStats['pitch_type'].astype(str)
allStats['p_throws']  = allStats['p_throws'].astype(str)
allStats['p_throws']

0          L
1          L
2          L
3          L
4          L
          ..
2847245    L
2847246    L
2847247    L
2847248    L
2847249    L
Name: p_throws, Length: 2847250, dtype: object

In [6]:
#allStats['stand'] = allStats['stand'].astype(str)
#allStats['top'] = allStats['top'].astype(str)
#allStats['type'] = allStats['type'].astype(str)
#allStats['p_throws']  = allStats['p_throws'].astype(str)

###Label encoding


In [5]:
cLB = LabelBinarizer()
cLB.fit(allStats['code'])
cLB.classes_
er=cLB.transform(allStats['code'])
allStats['codeOut'] = er.tolist()

tLB = LabelEncoder()


ty = pd.DataFrame(tLB.fit_transform(allStats['type']))#, #columns=tLB.classes_)

pLB = LabelBinarizer()


r1 = pd.DataFrame(pLB.fit_transform(allStats['pitch_type']), columns=pLB.classes_)

allStats=pd.merge(allStats, r1, left_index=True, right_index = True)

sLE = preprocessing.LabelEncoder()
sLE.fit(allStats['stand'])
allStats['stand'] = sLE.transform(allStats['stand'])

tLE = preprocessing.LabelEncoder()
tLE.fit(allStats['top'])
allStats['top'] = tLE.transform(allStats['top'])


eLE = preprocessing.LabelEncoder()
eLE.fit(allStats['p_throws'])
allStats['p_throws'] = eLE.transform(allStats['p_throws'])


allStats



,start_speed,end_speed,spin_rate,spin_dir,break_y,ax,ay,az,vx0,vy0,...,FS,FT,IN,KC,KN,PO,SC,SI,SL,UN
0,92.9,84.1,2305.052,159.235,23.7,7.665000,34.685000,-11.960000,-6.409000,-136.065000,...,0,0,0,0,0,0,0,0,0,0
1,92.8,84.1,2689.935,151.402,23.7,12.043000,34.225000,-10.085000,-8.411000,-135.690000,...,0,0,0,0,0,0,0,0,0,0
2,94.1,85.2,2647.972,145.125,23.7,14.368000,35.276000,-11.560000,-9.802000,-137.668000,...,0,0,0,0,0,0,0,0,0,0
3,91.0,84.0,1289.590,169.751,23.8,2.104000,28.354000,-20.540000,-8.071000,-133.005000,...,0,0,0,0,0,0,0,0,0,0
4,75.4,69.6,1374.569,280.671,23.8,-10.280000,21.774000,-34.111000,-6.309000,-110.409000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847245,83.9,78.2,608.594,275.800,23.9,-5.097492,20.344010,-32.691751,-6.895539,-122.140540,...,0,0,0,0,0,0,0,0,1,0
2847246,97.7,89.5,2262.907,155.009,23.8,9.344343,29.592944,-12.127944,-12.912992,-141.795182,...,0,0,0,0,0,0,0,0,0,0
2847247,97.3,89.6,2514.010,147.166,23.8,13.292081,30.048939,-11.576431,-12.095230,-141.465465,...,0,0,0,0,0,0,0,0,0,0
2847248,95.9,88.6,2318.775,144.921,23.8,12.786338,28.690698,-13.967178,-11.422113,-139.199258,...,0,0,0,0,0,0,0,0,0,0


###Heatmap

In [8]:
ty

,0,1,2
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,1,0,0
...,...,...,...
2847245,0,0,1
2847246,1,0,0
2847247,0,1,0
2847248,0,1,0


###Normalization

In [6]:
allStats.drop(["code", 'type', 'pitch_type'], axis=1, inplace=True)
allStats.drop(['event'], axis=1, inplace=True)
#allStats.drop(['type'], axis=1, inplace=True)
allStats.drop(['codeOut'], axis=1, inplace=True)
#allStats.dtypes


min_max_scaler = preprocessing.MinMaxScaler()
#allStats.dtypes
x_scaled = min_max_scaler.fit_transform(allStats)

#correlation(pd.DataFrame(x_scaled))
#allStats = pd.DataFrame(x_scaled)

In [ ]:
#correlation(allStats)
print(min_max_scaler.get_params(True))
min_max_scaler.fit(allStats)
print(min_max_scaler.get_params(True))
min_max_scaler.get_feature_names_out()

###Trying to create testset and data set

In [12]:
from sklearn.model_selection import train_test_split

msk = np.random.rand(len(x_scaled)) <= 0.7
X_train, X_test, y_train, y_test = train_test_split(x_scaled, ty, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.3)


def MultiModel(dimsi):
  return keras.Sequential([
    keras.layers.Dense(60,input_dim=(dimsi), activation = 'relu'),
    keras.layers.Dense(50, activation = 'ReLU'),
    keras.layers.Dropout(.2, input_shape=(50,)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = 'tanh'),
    keras.layers.Dense(100, activation = 'linear'),
    keras.layers.Dense(50, activation = 'ReLU'),
    keras.layers.Dropout(.2, input_shape=(50,)),
    keras.layers.Dense(12, activation = 'sigmoid'),
    keras.layers.Dense(3, activation='softmax')
  ])






###Creating Tensors


In [14]:
def basicModel(dimsi):
  return keras.Sequential([
    keras.layers.Dense(60,input_dim=49, activation = 'relu'),
    keras.layers.Dropout(.2, input_shape=(50,)),
    keras.layers.Dense(50, activation = 'ReLU'),
    keras.layers.Dense(50, activation = 'ReLU'),
    keras.layers.Dropout(.2, input_shape=(50,)),
   keras.layers.Dense(12, activation = 'ReLU'),
     keras.layers.Dense(3, activation='softmax')
  ])

def sigmoidModel(dimsi):
 return keras.Sequential([
   keras.layers.Dense(60,input_dim=49, activation = 'sigmoid'),
   keras.layers.Dropout(.2, input_shape=(50,)),
   keras.layers.BatchNormalization(),
   keras.layers.Dense(50, activation = 'sigmoid'),
   keras.layers.Dense(50, activation = 'sigmoid'),
   keras.layers.Dropout(.2, input_shape=(50,)),
   keras.layers.Dense(12, activation = 'sigmoid'),
   keras.layers.Dense(3, activation='softmax')
  ])

def modelCompileAndRun():
  model = MultiModel(49)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =0.002),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              #loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  history = model.fit(X_train, y_train, epochs=50, batch_size=450, validation_data=(X_test, y_test))
  test_loss, test_acc = model.evaluate(X_test, y_test)
  createPlot(history)
  return model

model = modelCompileAndRun()
#model.load_weights('/content/drive/MyDrive/MLB Data/Weights')
test_loss, test_acc = model.evaluate(X_test, y_test)
model.save("/content/drive/MyDrive/MLB Data/modelSigmoid.keras")
y_test
allStats.shape


Epoch 1/50
4430/4430 [==============================] - 56s 12ms/step - loss: 0.7186 - accuracy: 0.6717 - val_loss: 0.6967 - val_accuracy: 0.6787
Epoch 2/50
4430/4430 [==============================] - 48s 11ms/step - loss: 0.6817 - accuracy: 0.6876 - val_loss: 0.6707 - val_accuracy: 0.6885
Epoch 3/50
4430/4430 [==============================] - 50s 11ms/step - loss: 0.6760 - accuracy: 0.6891 - val_loss: 0.6916 - val_accuracy: 0.6832
Epoch 4/50
 328/4430 [=>............................] - ETA: 51s - loss: 0.6749 - accuracy: 0.6903

KeyboardInterrupt: ignored

In [14]:
#history = model.fit(X_train, y_train, epochs=50, batch_size=450,validation_data=(X_test, y_test))
#test_loss, test_acc = model.evaluate(X_test, y_test)
#test_loss
model.save("/content/drive/MyDrive/MLB Data/modelSigmoid.keras")

In [ ]:
from sklearn.metrics import classification_report

#model = tf.keras.models.load_model('/content/drive/MyDrive/MLB Data/model.keras')
predictions = pd.DataFrame(model.predict(X_val))
m = np.zeros_like(predictions.values)
m[np.arange(len(predictions)), predictions.values.argmax(1)] = 1

predictions = pd.DataFrame(m, columns = ('B','S','X')).astype(int)
print (predictions)
print(y_val)
print(classification_report(y_val, predictions))


In [1]:
def createPlot(history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [14]:
from sklearn.metrics import accuracy_score
#result = model.predict(X_test)
#result = result.round();
#acc = accuracy_score(y_test, result)
#acc
#result[0]
#model.save('/content/drive/MyDrive/models/mod')


###K-nearest neighbour

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

KNN_model = KNeighborsClassifier(n_neighbors=5)

KNN_model.fit(X_train, y_train)
KNN_pred = KNN_model.predict(X_val[:500])
print(classification_report(KNN_pred, y_val[:500]))

              precision    recall  f1-score   support

           0       0.50      0.78      0.61       116
           1       0.70      0.64      0.67       255
           2       0.22      0.47      0.30        43

   micro avg       0.54      0.66      0.60       414
   macro avg       0.48      0.63      0.53       414
weighted avg       0.60      0.66      0.61       414
 samples avg       0.54      0.54      0.54       414



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Create svm model

In [15]:

from sklearn.metrics import classification_report
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score

y_val_flat = y_train.values.ravel()
svcModel = svm.SVC(kernel='rbf', degree=3, max_iter=30, C=1).fit(X_train, y_val_flat)
pred= svcModel.predict(X_test)
print(classification_report(pred, y_test.values.ravel()))
y_test_flat = y_test.values.ravel()


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.88      0.36      0.51    528852
           1       0.09      0.48      0.15     52030
           2       0.02      0.14      0.04     17040

    accuracy                           0.37    597922
   macro avg       0.33      0.33      0.24    597922
weighted avg       0.79      0.37      0.47    597922



In [18]:
accuracy = accuracy_score(y_test_flat, pred)
f1 = f1_score(y_test_flat, pred, average='weighted')

print('Accuracy SVC poly: ', "%.2f" % (accuracy*100))
print('Accuracy f1 poly: ', f1)


Accuracy SVC poly:  44.94
Accuracy f1 poly:  0.3172487960223621


In [ ]:
createSVM()

###Create basic model


In [ ]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())

###First test predictions


In [ ]:
untrained_preds = model_0(pitch_tensor)
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(outcome_tensor)}, Shape: {outcome_tensor.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 results:\n{outcome_tensor[:10]}")

##Setting loss function and optimizer


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
def correlation(df):
  pear_corr=df.corr(method='pearson')
  fig, ax = plt.subplots(figsize=(df.shape[1],df.shape[1]))
  im = ax.imshow(pear_corr, interpolation='nearest')
  fig.colorbar(im, orientation='vertical', fraction = 0.05)

# Show all ticks and label them with the dataframe column name
  ax.set_xticklabels(df.columns, rotation=65, fontsize=15)
  ax.set_yticklabels(df.columns, rotation=0, fontsize=15)

# Loop over data dimensions and create text annotations
  for i in range(len(df.columns)-1):
    for j in range(len(df.columns)-1):
      text = ax.text(j, i, round(pear_corr.to_numpy()[i, j], 2),
                   ha="center", va="center", color="black")

  plt.show()


https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html